# SGD Algorithm to predict movie ratings

<pre>
1. Download the data from <a href='https://drive.google.com/open?id=1-1z7iDB52cB6_JpO7Dqa-eOYSs-mivpq'> here </a>
2. the data will be of this formate, each data point is represented as a triplet of user_id, movie_id and rating 
<table>
<tr><th>user_id</th><th>movie_id</th><th>rating</th></tr>
<tr><td>77</td><td>236</td><td>3</td></tr>
<tr><td>471</td><td>208</td><td>5</td></tr>
<tr><td>641</td><td>401</td><td>4</td></tr>
<tr><td>31</td><td>298</td><td>4</td></tr>
<tr><td>58</td><td>504</td><td>5</td></tr>
<tr><td>235</td><td>727</td><td>5</td></tr>
</table>
<h3>task 1: Predict the rating for a given (user_id, movie_id) pair</h3>
</pre>
<ul>
<li><span class="math">\(\mu\)</span> : scalar mean rating</li>
<li><span class="math">\(b_i\)</span> : scalar bias term for user <span class="math">\(i\)</span></li>
<li><span class="math">\(c_j\)</span> : scalar bias term for movie <span class="math">\(j\)</span></li>
<li><span class="math">\(u_i\)</span> : K-dimensional vector for user <span class="math">\(i\)</span></li>
<li><span class="math">\(v_j\)</span> : K-dimensional vector for movie <span class="math">\(j\)</span></li>
</ul>
then the predicted rating $\hat{y}_{ij}$ for user i, movied j pair is calcuated as $\hat{y}_{ij} = \mu + b_i + c_j + u_i^T v_j$ here we will be finding the best values of $b_{i}$ and $c_{j}$ using SGD algorithm with the optimization problem for N users and M movies is defined as


$$
L = \min_{ b, c, \{ u_i \}_{i=1}^N, \{ v_j \}_{j=1}^M}
\quad
\alpha \Big(
    \sum_{j} \sum_{k} v_{jk}^2 
    + \sum_{i} \sum_{k} u_{ik}^2 
    + \sum_{i} b_i^2
    + \sum_{j} c_i^2
    \Big)
+ \sum_{i,j \in \mathcal{I}^{\text{train}}}
    (y_{ij} - \mu - b_i - c_j - u_i^T v_j)^2
$$

### TASK: 1
__SGD Algorithm to minimize the loss__
1. for each unique user initilize a bias value $B_{i}$ randomly, so if we have $N$ users $B$ will be a $N$ dimensional vector, the $i^{th}$ value of the $B$ will corresponds to the bias term for $i^{th}$ user

2. for each unique movie initilize a bias value $C_{j}$ randomly, so if we have $M$ movies $C$ will be a $M$ dimensional vector, the $j^{th}$ value of the $C$ will corresponds to the bias term for $j^{th}$ movie

3. Construct adjacency matrix with the given data, assumeing its  <a href='https://en.wikipedia.org/wiki/Bipartite_graph'> weighted un-directed bi-partited graph</a> and the weight of each edge is the rating given by user to the movie
<img src='https://i.imgur.com/rmUCGMb.jpg' width=200>
you can construct this matrix like $A[i][j]=r_{ij}$ here $i$ is user_id, $j$ is movie_id and $r_{ij}$ is rating given by user $i$ to the movie $j$

4. we will Apply SVD decomposition on the Adjaceny matrix <a href='https://stackoverflow.com/a/31528944/4084039'>link1</a>, <a href='https://machinelearningmastery.com/singular-value-decomposition-for-machine-learning/'> link2</a> and get three matrices $U, \sum, V$ such that $U \times \sum \times V^T = A$, <br> 
if $A$ is of dimensions $N \times M$ then <br>
U is of $N \times k$, <br>
$\sum$ is of $k \times k$ and <br>
$V$ is $M \times k$ dimensions. <br>

5. So the matrix $U$ can be represented as matrix representation of users, where each row $u_{i}$ represents a k-dimensional vector for a user

6. So the matrix $V$ can be represented as matrix representation of movies, where each row $v_{j}$ represents a k-dimensional vector for a movie

7. $\mu$ represents the mean of all the rating given in the dataset
</pre>

<br>8.
<code>
for each epoch:
    for each pair of (user, movie):
        b_i =  b_i - learning_rate * dL/db_i
        c_j =  c_j - learning_rate * dL/dc_j
    predict the ratings with formula </code> $\hat{y}_{ij} = \mu + b_i + c_j + \text{dot_product}(u_i , v_j)$
    <code>
    print the mean squared error with predicted ratings
    </code>

9. you can choose any learning rate and regularization term in the range $10^{-3}  \text{ to } 10^2$  <br>

10. __bonus__: instead of using SVD decomposition you can learn the vectors $u_i$, $v_j$ with the help of SGD algo similar to $b_i$ and $c_j$ 
### TASK: 2

As we know U is the learned matrix of user vectors, with its i-th row as the vector ui for user i. Each row of U can be seen as a "feature vector" for a particular user.

The question we'd like to investigate is this: do our computed per-user features that are optimized for predicting movie ratings contain anything to do with gender?

The provided data file <a href='https://drive.google.com/open?id=1PHFdJh_4gIPiLH5Q4UErH8GK71hTrzlY'>user_info.csv</a> contains an is_male column indicating which users in the dataset are male. Can you predict this signal given the features U?


> __Note 1__ : there is no train test split in the data, the goal of this assignment is to give an intution about how to do matrix factorization with the help of SGD and application of truncated SVD. for better understanding of the collabarative fillerting please check netflix case study. <br><br>
> __Note 2__ : Check if scaling of $U$, $V$ matrices improve the metric 

In [1]:
import pandas as pd
from tqdm import tqdm
data = pd.read_csv('C:\\Users\\user\\Desktop\\RecommendationSystem_TruncatedSVD\\ratings_train.csv')
data.shape

(89992, 3)

In [94]:
#print(len(data))
A=[]
for i in range(max(data.user_id)+1):
    a=[]
    for j in range(max(data.item_id)+1):
        p=[0]
        a.extend(p)    
    A.append(a) 

In [96]:
print(len(A))
print(len(A[0]))

943
1681


In [95]:
for i in range(len(data)):
    #print(i)
    A[data.loc[:,'user_id'][i]][data.loc[:,'item_id'][i]]=data.loc[:,'rating'][0]
    

In [103]:
b = np.random.normal(0, 0.1, size=(max(data.user_id)+1))
c = np.random.normal(0, 0.1, size=(max(data.item_id)+1))

In [105]:
from sklearn.utils.extmath import randomized_svd
import numpy as np 
matrix=np.asarray(A)
U, Sigma, VT = randomized_svd(matrix, n_components=5,n_iter=5, random_state=None)
print(U.shape)
print(Sigma.shape)
print(VT.T.shape)

(943, 5)
(5,)
(1681, 5)


In [116]:
MYU=data.loc[:,"rating"].mean()

In [ ]:
epoch=100
learning_rate=0.1

In [ ]:
for L in range(epoch):
    for K in range(len(data)):
        b[data.loc[:,'user_id'][K]] =  b[data.loc[:,'user_id'][K]] - learning_rate * dL/db_i
        c[data.loc[:,'item_id'][K]] =  c[data.loc[:,'item_id'][K]]- learning_rate * dL/dc_j
    predict the ratings with formula   𝑦̂ 𝑖𝑗=𝜇+𝑏𝑖+𝑐𝑗+dot_product(𝑢𝑖,𝑣𝑗)  
    print the mean squared error with predicted ratings

In [120]:
data_1 = pd.read_csv('C:\\Users\\user\\Desktop\\RecommendationSystem_TruncatedSVD\\user_info.txt')
data_1.shape

(943, 4)

In [121]:
data_1.loc[:,'is_male']

0      1
1      0
2      1
3      1
4      0
      ..
938    0
939    1
940    1
941    0
942    1
Name: is_male, Length: 943, dtype: int64

In [124]:
from sklearn import linear_model
X=np.asarray(U)
y=data_1.loc[:,'is_male']
clf = linear_model.SGDClassifier(eta0=0.0001, alpha=0.0001, loss='log', random_state=15, penalty='l2', tol=1e-3, verbose=2, learning_rate='constant')
clf
clf.fit(X=X, y=y)
clf.coef_, clf.coef_.shape, clf.intercept_

-- Epoch 1
Norm: 0.00, NNZs: 5, Bias: 0.019617, T: 943, Avg. loss: 0.691101
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 0.00, NNZs: 5, Bias: 0.038766, T: 1886, Avg. loss: 0.687111
Total training time: 0.03 seconds.
-- Epoch 3
Norm: 0.00, NNZs: 5, Bias: 0.057481, T: 2829, Avg. loss: 0.683305
Total training time: 0.03 seconds.
-- Epoch 4
Norm: 0.00, NNZs: 5, Bias: 0.075759, T: 3772, Avg. loss: 0.679673
Total training time: 0.07 seconds.
-- Epoch 5
Norm: 0.00, NNZs: 5, Bias: 0.093617, T: 4715, Avg. loss: 0.676209
Total training time: 0.08 seconds.
-- Epoch 6
Norm: 0.00, NNZs: 5, Bias: 0.111069, T: 5658, Avg. loss: 0.672902
Total training time: 0.10 seconds.
-- Epoch 7
Norm: 0.00, NNZs: 5, Bias: 0.128083, T: 6601, Avg. loss: 0.669749
Total training time: 0.11 seconds.
-- Epoch 8
Norm: 0.00, NNZs: 5, Bias: 0.144720, T: 7544, Avg. loss: 0.666742
Total training time: 0.11 seconds.
-- Epoch 9
Norm: 0.01, NNZs: 5, Bias: 0.160973, T: 8487, Avg. loss: 0.663871
Total training time: 0.12 se

(array([[ 0.01537419,  0.00374645,  0.00031084,  0.00290112, -0.00366014]]),
 (1, 5),
 array([0.48355292]))